In [1]:
from keras_dec import DeepEmbeddingClustering
import numpy as np
import pandas as pd

Using TensorFlow backend.
/home/eimantaszaranka/miniconda3/envs/mt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eimantaszaranka/miniconda3/envs/mt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eimantaszaranka/miniconda3/envs/mt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:
labels = pd.read_csv('encoded_labels_df.csv')

# Bert Embeddings

In [3]:
data = pd.read_csv('bert_embeddings.csv')

In [4]:
c_20 = DeepEmbeddingClustering(n_clusters=20, input_dim=data.shape[1], pretrained_weights='bert_autoencoder.h5')
c_20.initialize(data.to_numpy(), save_autoencoder=True, name='bert_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.


In [5]:
y_pred = c_20.cluster(data.to_numpy(), y=labels['class_20'].to_numpy(), max_iter=1e5)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.20256
Iteration 109, Accuracy 0.21748
Iteration 218, Accuracy 0.24039
Iteration 327, Accuracy 0.25095
Iteration 436, Accuracy 0.25827
Iteration 545, Accuracy 0.26448
Iteration 654, Accuracy 0.26883
Iteration 763, Accuracy 0.26552
Iteration 872, Accuracy 0.27865
Iteration 981, Accuracy 0.28022
Iteration 1090, Accuracy 0.26684
Iteration 1199, Accuracy 0.28361
Iteration 1308, Accuracy 0.28347
Iteration 1417, Accuracy 0.29003
Iteration 1526, Accuracy 0.28846
Iteration 1635, Accuracy 0.29435
Iteration 1744, Accuracy 0.29221
Iteration 1853, Accuracy 0.29139
Iteration 1962, Accuracy 0.28811
Iteration 2071, Accuracy 0.28104
Iteration 2180, Accuracy 0.28339
Iteration 2289, Accuracy 0.28254
Iteration 2398, Accuracy 0.28343
Iteration 2507, Accuracy 0.28279
Iteration 2616, Accuracy 0.28893
Iteration 2725, Accuracy 0.27961
Iteration 2834, Accuracy 0.27883
Iteration 2943, Accuracy 0.28746
Iteration 3052, Accuracy 0.29121
Iteration 3161, 

In [4]:
c_82 = DeepEmbeddingClustering(n_clusters=82, input_dim=data.shape[1], pretrained_weights='bert_autoencoder.h5')
c_82.initialize(data.to_numpy(), save_autoencoder=True, name='bert_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_82.cluster(data.to_numpy(), y=labels['class_82'].to_numpy(), max_iter=1e5)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.12969
Iteration 109, Accuracy 0.1389
Iteration 218, Accuracy 0.14393
Iteration 327, Accuracy 0.14778
Iteration 436, Accuracy 0.147
Iteration 545, Accuracy 0.14792
Iteration 654, Accuracy 0.14835
Iteration 763, Accuracy 0.14114
Iteration 872, Accuracy 0.13843
Iteration 981, Accuracy 0.15092
Iteration 1090, Accuracy 0.14214
Iteration 1199, Accuracy 0.14354
Iteration 1308, Accuracy 0.15417
Iteration 1417, Accuracy 0.15421
Iteration 1526, Accuracy 0.15699
Iteration 1635, Accuracy 0.15139
Iteration 1744, Accuracy 0.16695
Iteration 1853, Accuracy 0.15877
Iteration 1962, Accuracy 0.16509
Iteration 2071, Accuracy 0.14817
Iteration 2180, Accuracy 0.15242
Iteration 2289, Accuracy 0.16627
Iteration 2398, Accuracy 0.17173
Iteration 2507, Accuracy 0.14803
Iteration 2616, Accuracy 0.16787
Iteration 2725, Accuracy 0.15003
Iteration 2834, Accuracy 0.1710

In [5]:
np.save('bert_82_pred.npy', y_pred)

In [6]:
c_595 = DeepEmbeddingClustering(n_clusters=595, input_dim=data.shape[1], pretrained_weights='bert_autoencoder.h5')
c_595.initialize(data.to_numpy(), save_autoencoder=True, name='bert_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_595.cluster(data.to_numpy(), y=labels['class_595'].to_numpy(), max_iter=1e5)
np.save('bert_595_pred.npy', y_pred)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.08269
Iteration 109, Accuracy 0.08454
Iteration 218, Accuracy 0.08968
Iteration 327, Accuracy 0.09903
Iteration 436, Accuracy 0.10403
Iteration 545, Accuracy 0.1026
Iteration 654, Accuracy 0.09721
Iteration 763, Accuracy 0.10164
Iteration 872, Accuracy 0.09571
Iteration 981, Accuracy 0.095
Iteration 1090, Accuracy 0.09739
Iteration 1199, Accuracy 0.09372
Iteration 1308, Accuracy 0.08644
Iteration 1417, Accuracy 0.09418
Iteration 1526, Accuracy 0.08408
Iteration 1635, Accuracy 0.07723
Iteration 1744, Accuracy 0.08001
Iteration 1853, Accuracy 0.08415
Iteration 1962, Accuracy 0.08454
Iteration 2071, Accuracy 0.0844
Iteration 2180, Accuracy 0.08626
Iteration 2289, Accuracy 0.07783
Iteration 2398, Accuracy 0.08151
Iteration 2507, Accuracy 0.0763
Iteration 2616, Accuracy 0.08187
Iteration 2725, Accuracy 0.07908
Iteration 2834, Accuracy 0.08026


In [12]:
c_769 = DeepEmbeddingClustering(n_clusters=769, input_dim=data.shape[1], pretrained_weights='bert_autoencoder.h5')
c_769.initialize(data.to_numpy(), save_autoencoder=True, name='bert_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.


In [13]:
y_pred = c_769.cluster(data.to_numpy(), y=labels['class_769'].to_numpy(), max_iter=1e5)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.08012
Iteration 109, Accuracy 0.08536
Iteration 218, Accuracy 0.08422
Iteration 327, Accuracy 0.08804
Iteration 436, Accuracy 0.08854
Iteration 545, Accuracy 0.09446
Iteration 654, Accuracy 0.09
Iteration 763, Accuracy 0.09329
Iteration 872, Accuracy 0.0905
Iteration 981, Accuracy 0.08219
Iteration 1090, Accuracy 0.08879
Iteration 1199, Accuracy 0.08372
Iteration 1308, Accuracy 0.09008
Iteration 1417, Accuracy 0.08651
Iteration 1526, Accuracy 0.09022
Iteration 1635, Accuracy 0.08101
Iteration 1744, Accuracy 0.08822
Iteration 1853, Accuracy 0.07969
Iteration 1962, Accuracy 0.08472
Iteration 2071, Accuracy 0.0808
Iteration 2180, Accuracy 0.08083
Iteration 2289, Accuracy 0.08126
Iteration 2398, Accuracy 0.07976
Iteration 2507, Accuracy 0.07866
Iteration 2616, Accuracy 0.07844
Iteration 2725, Accuracy 0.07691
Iteration 2834, Accuracy 0.07869
Iteration 2943, Accuracy 0.07965
Iteration 3052, Accuracy 0.07965
Iteration 3161, Accur

# LaBSE

In [7]:
data = pd.read_csv('labse_embeddings.csv')

In [19]:
c_20 = DeepEmbeddingClustering(n_clusters=20, input_dim=data.shape[1])
c_20.initialize(data.to_numpy(), save_autoencoder=True, name='labse_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

layerwise pretrain
Epoch 1/458
28021/28021 [==============================] - 1s 40us/step - loss: 0.0012
Epoch 2/458
28021/28021 [==============================] - 1s 37us/step - loss: 0.0011
Epoch 3/458
28021/28021 [==============================] - 1s 37us/step - loss: 9.9074e-04
Epoch 4/458
28021/28021 [==============================] - 1s 38us/step - loss: 9.4630e-04
Epoch 5/458
28021/28021 [==============================] - 1s 37us/step - loss: 9.2210e-04
Epoch 6/458
28021/28021 [==============================] - 1s 39us/step - loss: 9.0885e-04
Epoch 7/458
28021/28021 [==============================] - 1s 38us/step - loss: 9.0157e-04
Epoch 8/458
28021/28021 [==============================] - 1s 39us/step - loss: 8.9747e-04
Epoch 9/458
28021/28021 [==============================] - 1s 37us/step - loss: 8.9521e-04
Epoch 10/458
28021/28021 [==============================] - 1s 37us/step - loss: 8.9384e-04
Epoch 11/458
28021/28021 [==============================] - 1s 36us/step - los

In [22]:
y_pred = c_20.cluster(data.to_numpy(), y=labels['class_20'].to_numpy(), max_iter=1e5, tol=0.0001)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.12994
Iteration 109, Accuracy 0.1299
Iteration 218, Accuracy 0.1299
Iteration 327, Accuracy 0.1299
Iteration 436, Accuracy 0.1299
Iteration 545, Accuracy 0.12987
Iteration 654, Accuracy 0.1299
Iteration 763, Accuracy 0.1299
Iteration 872, Accuracy 0.12987
Iteration 981, Accuracy 0.12987
Iteration 1090, Accuracy 0.12987
Reached tolerance threshold. Stopping training.


In [8]:
c_82 = DeepEmbeddingClustering(n_clusters=82, input_dim=data.shape[1], pretrained_weights='labse_autoencoder.h5')
c_82.initialize(data.to_numpy(), save_autoencoder=True, name='labse_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_82.cluster(data.to_numpy(), y=labels['class_82'].to_numpy(), max_iter=1e5)
np.save('labse_82_pred.npy', y_pred)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.10264
Iteration 109, Accuracy 0.1026
Reached tolerance threshold. Stopping training.


In [9]:
c_595 = DeepEmbeddingClustering(n_clusters=595, input_dim=data.shape[1], pretrained_weights='labse_autoencoder.h5')
c_595.initialize(data.to_numpy(), save_autoencoder=True, name='labse_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_595.cluster(data.to_numpy(), y=labels['class_595'].to_numpy(), max_iter=1e5)
np.save('labse_595_pred.npy', y_pred)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.0869
Iteration 109, Accuracy 0.08697
Reached tolerance threshold. Stopping training.


In [27]:
c_769 = DeepEmbeddingClustering(n_clusters=769, input_dim=data.shape[1], pretrained_weights='labse_autoencoder.h5')
c_769.initialize(data.to_numpy(), save_autoencoder=True, name='labse_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.


In [28]:
y_pred = c_769.cluster(data.to_numpy(), y=labels['class_769'].to_numpy(), max_iter=1e5)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.08397
Iteration 109, Accuracy 0.08404
Reached tolerance threshold. Stopping training.


# W2V

In [11]:
data = pd.read_csv('w2v_embeddings.csv')

In [31]:
c_20 = DeepEmbeddingClustering(n_clusters=20, input_dim=data.shape[1])
c_20.initialize(data.to_numpy(), save_autoencoder=True, name='w2v_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

layerwise pretrain
Epoch 1/458
28021/28021 [==============================] - 1s 21us/step - loss: 0.0505
Epoch 2/458
28021/28021 [==============================] - 1s 20us/step - loss: 0.0217
Epoch 3/458
28021/28021 [==============================] - 1s 21us/step - loss: 0.0124
Epoch 4/458
28021/28021 [==============================] - 1s 20us/step - loss: 0.0090
Epoch 5/458
28021/28021 [==============================] - 1s 20us/step - loss: 0.0072
Epoch 6/458
28021/28021 [==============================] - 1s 22us/step - loss: 0.0060
Epoch 7/458
28021/28021 [==============================] - 1s 21us/step - loss: 0.0053
Epoch 8/458
28021/28021 [==============================] - 1s 20us/step - loss: 0.0048
Epoch 9/458
28021/28021 [==============================] - 1s 19us/step - loss: 0.0045
Epoch 10/458
28021/28021 [==============================] - 1s 22us/step - loss: 0.0042
Epoch 11/458
28021/28021 [==============================] - 1s 20us/step - loss: 0.0040
Epoch 12/458
28021/280

In [32]:
y_pred = c_20.cluster(data.to_numpy(), y=labels['class_20'].to_numpy(), max_iter=1e5)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.22273
Iteration 109, Accuracy 0.24514
Iteration 218, Accuracy 0.21388
Iteration 327, Accuracy 0.25331
Iteration 436, Accuracy 0.23104
Iteration 545, Accuracy 0.22187
Iteration 654, Accuracy 0.22176
Iteration 763, Accuracy 0.2218
Iteration 872, Accuracy 0.22465
Iteration 981, Accuracy 0.22333
Iteration 1090, Accuracy 0.22947
Iteration 1199, Accuracy 0.22169
Iteration 1308, Accuracy 0.22665
Iteration 1417, Accuracy 0.22212
Iteration 1526, Accuracy 0.23118
Iteration 1635, Accuracy 0.22579
Iteration 1744, Accuracy 0.23422
Iteration 1853, Accuracy 0.23729
Iteration 1962, Accuracy 0.23308
Iteration 2071, Accuracy 0.2405
Iteration 2180, Accuracy 0.24881
Iteration 2289, Accuracy 0.24592
Iteration 2398, Accuracy 0.24193
Iteration 2507, Accuracy 0.23522
Iteration 2616, Accuracy 0.24228
Iteration 2725, Accuracy 0.25013
Iteration 2834, Accuracy 0.25909
Iteration 2943, Accuracy 0.24239
Iteration 3052, Accuracy 0.25206
Iteration 3161, Ac

In [12]:
c_82 = DeepEmbeddingClustering(n_clusters=82, input_dim=data.shape[1], pretrained_weights='w2v_autoencoder.h5')
c_82.initialize(data.to_numpy(), save_autoencoder=True, name='w2v_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_82.cluster(data.to_numpy(), y=labels['class_82'].to_numpy(), max_iter=1e5)
np.save('w2v_82_pred.npy', y_pred)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.13747
Iteration 109, Accuracy 0.14974
Iteration 218, Accuracy 0.17373
Iteration 327, Accuracy 0.15328
Iteration 436, Accuracy 0.18604
Iteration 545, Accuracy 0.17266
Iteration 654, Accuracy 0.18654
Iteration 763, Accuracy 0.17112
Iteration 872, Accuracy 0.15253
Iteration 981, Accuracy 0.17419
Iteration 1090, Accuracy 0.15574
Iteration 1199, Accuracy 0.17234
Iteration 1308, Accuracy 0.16241
Iteration 1417, Accuracy 0.14907
Iteration 1526, Accuracy 0.16566
Iteration 1635, Accuracy 0.18458
Iteration 1744, Accuracy 0.1667
Iteration 1853, Accuracy 0.18079
Iteration 1962, Accuracy 0.18947
Iteration 2071, Accuracy 0.17059
Iteration 2180, Accuracy 0.18069
Iteration 2289, Accuracy 0.1668
Iteration 2398, Accuracy 0.16166
Iteration 2507, Accuracy 0.17016
Iteration 2616, Accuracy 0.19018
Iteration 2725, Accuracy 0.17473
Iteration 2834, Accuracy 0.196

In [13]:
c_595 = DeepEmbeddingClustering(n_clusters=595, input_dim=data.shape[1], pretrained_weights='w2v_autoencoder.h5')
c_595.initialize(data.to_numpy(), save_autoencoder=True, name='w2v_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)
y_pred = c_595.cluster(data.to_numpy(), y=labels['class_595'].to_numpy(), max_iter=1e5)
np.save('w2v_595_pred.npy', y_pred)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.
Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.09122
Iteration 109, Accuracy 0.10606
Iteration 218, Accuracy 0.09243
Iteration 327, Accuracy 0.10974
Iteration 436, Accuracy 0.12359
Iteration 545, Accuracy 0.11302
Iteration 654, Accuracy 0.10092
Iteration 763, Accuracy 0.0955
Iteration 872, Accuracy 0.10581
Iteration 981, Accuracy 0.09896
Iteration 1090, Accuracy 0.11306
Iteration 1199, Accuracy 0.10242
Iteration 1308, Accuracy 0.1111
Iteration 1417, Accuracy 0.10335
Iteration 1526, Accuracy 0.11063
Iteration 1635, Accuracy 0.10713
Iteration 1744, Accuracy 0.09657
Iteration 1853, Accuracy 0.1031
Iteration 1962, Accuracy 0.09693
Iteration 2071, Accuracy 0.10339
Iteration 2180, Accuracy 0.11202
Iteration 2289, Accuracy 0.11895
Iteration 2398, Accuracy 0.10239
Iteration 2507, Accuracy 0.11488
Iteration 2616, Accuracy 0.12034
Iteration 2725, Accuracy 0.12102
Iteration 2834, Accuracy 0.1236

In [34]:
c_769 = DeepEmbeddingClustering(n_clusters=769, input_dim=data.shape[1], pretrained_weights='w2v_autoencoder.h5')
c_769.initialize(data.to_numpy(), save_autoencoder=True, name='w2v_autoencoder', finetune_iters=100000, layerwise_pretrain_iters=50000)

Loading pretrained weights for autoencoder.
Initializing cluster centres with k-means.


In [35]:
y_pred = c_769.cluster(data.to_numpy(), y=labels['class_769'].to_numpy(), max_iter=1e5)

Update interval 109
Save interval 5472
Iteration 0, Accuracy 0.09029
Iteration 109, Accuracy 0.10392
Iteration 218, Accuracy 0.09332
Iteration 327, Accuracy 0.1015
Iteration 436, Accuracy 0.11941
Iteration 545, Accuracy 0.10738
Iteration 654, Accuracy 0.11909
Iteration 763, Accuracy 0.10738
Iteration 872, Accuracy 0.11973
Iteration 981, Accuracy 0.1137
Iteration 1090, Accuracy 0.1248
Iteration 1199, Accuracy 0.11548
Iteration 1308, Accuracy 0.10342
Iteration 1417, Accuracy 0.11534
Iteration 1526, Accuracy 0.12116
Iteration 1635, Accuracy 0.12448
Iteration 1744, Accuracy 0.12676
Iteration 1853, Accuracy 0.1258
Iteration 1962, Accuracy 0.12091
Iteration 2071, Accuracy 0.12316
Iteration 2180, Accuracy 0.11391
Iteration 2289, Accuracy 0.10588
Iteration 2398, Accuracy 0.10096
Iteration 2507, Accuracy 0.1031
Iteration 2616, Accuracy 0.09932
Iteration 2725, Accuracy 0.09639
Iteration 2834, Accuracy 0.09818
Iteration 2943, Accuracy 0.10888
Iteration 3052, Accuracy 0.09935
Iteration 3161, Accur

# Export

In [36]:
np.save('w2v_769_pred.npy', y_pred)